In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [ ]:
# Для вашего удобства списки с именами разных колонок

# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols + cat_cols
target_col = 'Churn'

In [ ]:
# Custom transformer which relaces spaces by zeros in TotalSpent column and
# converts it to floats. Returns pd.DataFrame
class TotalSpentTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # replace spaces by zeros
        SImp = SimpleImputer(missing_values = " ", fill_value = "0", strategy="constant")
        SImp.fit(X[["TotalSpent"]])
        new_TotalSpent = SImp.transform(X[["TotalSpent"]])

        # convert strings to float
        new_TotalSpent = new_TotalSpent.astype(float)

        # create new dataframe without changing input one
        newX = X.assign(**{"TotalSpent" : new_TotalSpent})
        return newX

def ClientPeriodTransformer(X):
    # replace spaces by zeros
    SImp = SimpleImputer(missing_values = 0, fill_value = 1, strategy="constant")
    SImp.fit(X[["ClientPeriod"]])
    new_ClientPeriod = SImp.transform(X[["ClientPeriod"]])

    new_TotalSpent = X[["TotalSpent"]].copy()
    new_TotalSpent.loc[X["TotalSpent"] == 0, "TotalSpent"] = X[X["TotalSpent"] == 0]["MonthlySpending"]

    newX = X.assign(**{"TotalSpent" : new_TotalSpent, "ClientPeriod": new_ClientPeriod})
    return newX

def add_spend_ratio(X):
    newX = X.assign(**{"spend_ratio" : X["TotalSpent"] / X["MonthlySpending"]})
    return newX

def add_monthly_avg(X):
    newX = X.assign(**{"MonthlyAvg" : X["TotalSpent"] / X["ClientPeriod"]})
    return newX

def add_last_more_than_avg(X):
    newX = X.assign(**{"LastMoreThanAvg" : (X["MonthlySpending"] / X["MonthlyAvg"]).astype(int)})
    return newX

def add_MonthlySpendingLog(X):
    newX = X.assign(**{"MonthlySpendingLog" : np.log(X["MonthlySpending"])})
    return newX

def add_TotalSpentLog(X):
    newX = X.assign(**{"TotalSpentLog" : np.log(X["TotalSpent"])})
    return newX

def remove_TotalSpent_MonthlySpending(X):
    old_cols = X.columns
    new_cols = [c for c in old_cols if c not in ["TotalSpent", "MonthlySpending"]]
    newX = X[new_cols]
    return newX

def myOneHotEncoderFunc(X, cat_cols=None):
    ohe = OneHotEncoder(sparse=False)
    encodedX = ohe.fit_transform(X[cat_cols])
    encoded_cols = ohe.get_feature_names(cat_cols)
    encodedX_df = pd.DataFrame(encodedX, columns=encoded_cols, index=X.index)
    
    non_cat_cols = [col for col in X.columns if col not in cat_cols]
    
    encoded_cols_filtered = []
    for col in cat_cols:
        encoded_cols_filtered.extend([c for c in encoded_cols if c.startswith(col + "_")][:-1])
    newX = pd.concat([X[non_cat_cols], encodedX_df[encoded_cols_filtered]], axis=1)
    return newX

def myOneHotEncoderTransformer(cat_cols):
    return FunctionTransformer(myOneHotEncoderFunc, kw_args={"cat_cols":cat_cols})

def print_roc_auc_score(model, X, y):
    roc_aucs = cross_val_score(model, X, y, cv=5, scoring="roc_auc", n_jobs=-1)
    print(f"scores: {roc_aucs}")
    print(f"avg: {roc_aucs.mean()}")
    print(f"std: {roc_aucs.std()}")

In [ ]:
# Load data and split into train/test
data = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/train.csv')
X = data[feature_cols]
y = data[target_col]

In [ ]:
# Combine all default data transformations into single pipeline
from sklearn.pipeline import Pipeline

preprocessing_pipeline = Pipeline(
    [("TotalSpent", TotalSpentTransformer()),
     ("SpendRatio", FunctionTransformer(add_spend_ratio)),

     ("ClientPeriod", FunctionTransformer(ClientPeriodTransformer)),
     
     ("MonthlyAvg", FunctionTransformer(add_monthly_avg)),
     ("LastMoreThanAvg", FunctionTransformer(add_last_more_than_avg)),

     ("MonthlySpendingLog", FunctionTransformer(add_MonthlySpendingLog)),
     ("TotalSpentLog", FunctionTransformer(add_TotalSpentLog)),
    ]
)

num_cols = num_cols + ["spend_ratio"]


# IsolationForest

In [ ]:
if_pipeline = make_pipeline(
    preprocessing_pipeline,
    myOneHotEncoderTransformer(cat_cols),
    IsolationForest(contamination=0.02)
)
if_pipeline.fit(X)
anomaly_scores = if_pipeline.predict(X)

In [ ]:
anomaly_df = pd.DataFrame(anomaly_scores, columns=["anomaly"])

A = pd.concat([y, anomaly_df, X], axis=1)
A[A["anomaly"] == -1]

# X=X[anomaly_scores==1]
# y=y[anomaly_scores==1]


# Logistic regression

In [ ]:
logreg_pipeline = make_pipeline(
    preprocessing_pipeline,
    myOneHotEncoderTransformer(cat_cols),
    StandardScaler(),
#     PCA(n_components=23),
    
    LogisticRegression(penalty="l2", C=10, l1_ratio=None, random_state=2, solver="saga", max_iter=10000),
)

print_roc_auc_score(logreg_pipeline, X, y)

In [ ]:
# Testing logreg params
params = [
    {
#         "logisticregression__C": [100, 10, 1, 0.1, 0.01, 0.001],
#         "logisticregression__penalty": ["l2", "none"],
        "logisticregression__class_weight": ["balanced", None],
#         "logisticregression__l1_ratio": [None]
    },

#     {
#         "logisticregression__C": [100, 10, 1, 0.1, 0.01, 0.001],
#         "logisticregression__penalty": ["elasticnet"],
#         "logisticregression__class_weight": ["balanced", None],
#         "logisticregression__l1_ratio": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#     }
]


logreg_grid = GridSearchCV(logreg_pipeline, params, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
logreg_grid.fit(X, y)
print(logreg_grid.best_params_)
logreg_grid.cv_results_['mean_test_score']

# Naive Bayes

In [ ]:
nb_pipeline = make_pipeline(
    preprocessing_pipeline,
    myOneHotEncoderTransformer(cat_cols),
    StandardScaler(),
    GaussianNB()
)

print_roc_auc_score(nb_pipeline, X, y)

# SVC

In [ ]:
svc_pipeline = make_pipeline(
    preprocessing_pipeline,
    myOneHotEncoderTransformer(cat_cols),
    StandardScaler(),
    SVC(probability=True, C=0.5, kernel="poly", gamma="scale", class_weight="balanced", degree=1)
)

print_roc_auc_score(svc_pipeline, X, y)

In [ ]:
params = [{
#                "svc__C": [10, 5, 1, 0.5, 0.1, 0.05, 0.01, 0.005],
#                "svc__kernel": ["linear", "rbf", "sigmoid"],
#                "svc__gamma": ["scale", "auto"],
               "svc__class_weight": ["balanced", None],
          },

#           {
#                "svc__C": [10, 5, 1, 0.5, 0.1, 0.05, 0.01, 0.005],
#                "svc__kernel": ["poly"],
#                "svc__gamma": ["scale", "auto"],
#                "svc__class_weight": ["balanced", None],
#                "svc__degree": [1, 2, 3, 4]
#           }
]

logreg_grid = GridSearchCV(svc_pipeline, params, cv=3, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
logreg_grid.fit(X, y)
print(logreg_grid.best_params_)
logreg_grid.cv_results_['mean_test_score']

# CatBoost

In [ ]:
catboost_pipeline = make_pipeline(
    preprocessing_pipeline,
    CatBoostClassifier(iterations=400,
                       depth=4,
                       learning_rate=0.05,
                       loss_function='Logloss',
                       verbose=False,
                       cat_features=cat_cols,
                       random_seed = 4,
                       l2_leaf_reg = 40,
                       eval_metric='AUC')
)

print_roc_auc_score(catboost_pipeline, X, y)

In [ ]:
params = {
    "catboostclassifier__depth": [4, 6, 8, 10],
#     "catboostclassifier__learning_rate": [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.5, 1],
#     "catboostclassifier__l2_leaf_reg": [0.1, 0.5, 1, 10, 20, 40, 60],
#     "catboostclassifier__iterations": [100, 200, 500],
#     "catboostclassifier__random_seed": [1, 2, 3, 4, 5, 6, 7, 8, 9], # test random seed just for fun =)
}

cb_grid = GridSearchCV(catboost_pipeline, params, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
cb_grid.fit(X, y)
print(cb_grid.best_params_)
cb_grid.cv_results_['mean_test_score']

# Предсказания

# XGBoost

In [ ]:
xgb_pipeline = make_pipeline(
    preprocessing_pipeline,
    myOneHotEncoderTransformer(cat_cols),
    XGBClassifier(n_jobs=-1,
                  use_label_encoder=False,
                  learning_rate=0.1,
                  max_depth=1,
                  n_estimators=200)

)

print_roc_auc_score(xgb_pipeline, X, y)

In [ ]:
params = {
#     "xgbclassifier__n_estimators": [50, 100, 200, 500, 1000],
#     "xgbclassifier__max_depth": [1, 2, 4, 6, 8, 10],
#     "xgbclassifier__learning_rate": [0.001, 0.1, 1, 10],

    "xgbclassifier__max_depth": [1, 2],
}

dt_grid = GridSearchCV(xgb_pipeline, params, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
dt_grid.fit(X, y)
print(dt_grid.best_params_)
dt_grid.cv_results_['mean_test_score']

# DecisionTreeClassifier

In [ ]:
dtree_pipeline = make_pipeline(
    preprocessing_pipeline,
    myOneHotEncoderTransformer(cat_cols),
    DecisionTreeClassifier(min_samples_leaf=15, max_depth=5, random_state=2)
)

print_roc_auc_score(dtree_pipeline, X, y)

In [ ]:
params = {
    "decisiontreeclassifier__min_samples_leaf": [1, 5, 10, 15, 20],
    "decisiontreeclassifier__max_depth": [5, 10, 15, 20, 100],
}

dt_grid = GridSearchCV(dtree_pipeline, params, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
dt_grid.fit(X, y)
print(dt_grid.best_params_)
dt_grid.cv_results_['mean_test_score']

# RandomForestClassifier

In [ ]:
random_forest_pipeline = make_pipeline(
    preprocessing_pipeline,
    myOneHotEncoderTransformer(cat_cols),
#     PCA(n_components=23),
    RandomForestClassifier(n_estimators=200, min_samples_leaf=20, max_depth=100, random_state=2,
                           class_weight="balanced",
                           criterion="entropy"
                          )
)

print_roc_auc_score(random_forest_pipeline, X, y)

In [ ]:
params = {
#     "randomforestclassifier__n_estimators": [20, 50, 100, 200, 500],
#     "randomforestclassifier__min_samples_leaf": [1, 5, 10, 15, 20, 25, 30],
#     "randomforestclassifier__max_depth": [5, 10, 15, 20, 100],
    "randomforestclassifier__criterion": ["gini", "entropy"],
#     "randomforestclassifier__class_weight" : ["balanced", "balanced_subsample", None]
}

rf_grid = GridSearchCV(random_forest_pipeline, params, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
rf_grid.fit(X, y)
print(rf_grid.best_params_)
rf_grid.cv_results_['mean_test_score']

# KNeighborsClassifier

In [ ]:
knn_pipeline = make_pipeline(
    preprocessing_pipeline,
    myOneHotEncoderTransformer(cat_cols),
    PCA(n_components=23),
    StandardScaler(),
    
    KNeighborsClassifier(n_neighbors=40, weights="uniform", p=1, metric='manhattan')
)

print_roc_auc_score(knn_pipeline, X, y)

In [ ]:
params = {
#     "kneighborsclassifier__n_neighbors": [1,2, 4, 6, 8, 10, 15, 20, 30, 40, 50, 70, 100],
    "kneighborsclassifier__weights": ["uniform", "distance"],
#     "kneighborsclassifier__p": [1, 2],
#     "kneighborsclassifier__metric": ["euclidean", "manhattan", "chebyshev", "minkowski"]
}

rf_grid = GridSearchCV(knn_pipeline, params, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
rf_grid.fit(X, y)
print(rf_grid.best_params_)
rf_grid.cv_results_['mean_test_score']

# Stacking

In [ ]:
estimators = [("logreg", logreg_pipeline),
              ("dtree", dtree_pipeline),
#               ("rf", random_forest_pipeline),
#               ("cb", catboost_pipeline),
#               ("knn", knn_pipeline),
              ("nb", nb_pipeline),
#               ("svc", svc_pipeline),
#               ("xgb", xgb_pipeline),
             ]
final_estimator = LogisticRegression(max_iter=10000, solver="saga", C=1)

stacking = StackingClassifier(estimators=estimators, 
                              final_estimator=final_estimator,
                              cv=5,
                              stack_method="predict_proba",
                              passthrough=False)

print_roc_auc_score(stacking, X, y)

In [ ]:
params = {
    "logreg__logisticregression__C": [100, 10, 1, 0.1, 0.01, 0.001],
#     "logreg__logisticregression__penalty": ["l1", "l2", "elasticnet", "none"],
#     "logreg__logisticregression__l1_ratio": [0.1, 0.5, 0.9],

    "dtree__decisiontreeclassifier__min_samples_leaf": [1, 5, 10, 20],
#     "dtree__decisiontreeclassifier__max_depth": [5, 10, 15, 20, 100],

#     "rf__randomforestclassifier__n_estimators": [50, 100, 200, 500, 1000],
#     "rf__randomforestclassifier__min_samples_leaf": [1, 5, 10, 20, 30],
#     "rf__randomforestclassifier__max_depth": [5, 10, 15, 20, 100],
#     "rf__randomforestclassifier__criterion": ["gini", "entropy"],
#     "rf__randomforestclassifier__class_weight" : ["balanced", "balanced_subsample", None],

#     (n_estimators=200, min_samples_leaf=30, max_depth=15, 
#     "rf__randomforestclassifier__n_estimators": [100, 200, 500, 1000],
#     "rf__randomforestclassifier__min_samples_leaf": [1, 10, 30],
#     "rf__randomforestclassifier__max_depth": [5, 10, None],
#     "rf__randomforestclassifier__criterion": ["gini"],
#     "rf__randomforestclassifier__class_weight" : ["balanced", "balanced_subsample", None],

    
#     "cb__catboostclassifier__depth": [2, 4, 6, 8, 10],
#     "cb__catboostclassifier__learning_rate": [0.05, 0.1, 0.5, 1],
#     "cb__catboostclassifier__l2_leaf_reg": [0.1, 1, 10, 30, 60],
#     "cb__catboostclassifier__iterations": [50, 100, 200, 500, 1000],

#     "knn__kneighborsclassifier__n_neighbors": [1, 2, 3, 4, 5, 10, 20, 30, 50],
#     "knn__kneighborsclassifier__weights": ["uniform", "distance"],
#     "knn__kneighborsclassifier__p": [1, 2],
#     "knn__kneighborsclassifier__metric": ["euclidean", "manhattan", "chebyshev", "minkowski"],

    
#     "svc__svc__C": [1, 0.1, 0.01, 0.001],
#     "svc__svc__kernel": ["linear", "rbf", "sigmoid", "poly"],
#     "svc__svc__class_weight": ["balanced", None],

#     "xgb__xgbclassifier__n_estimators": [50, 100, 200, 500, 1000],
#     "xgb__xgbclassifier__max_depth": [2, 4, 6, 8, 10],
#     "xgb__xgbclassifier__learning_rate": [0.05, 0.1, 1, 10],
}


rf_grid = GridSearchCV(stacking, params, cv=3, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
# rf_grid = RandomizedSearchCV(stacking, params, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
rf_grid.fit(X, y)
print(rf_grid.best_params_)
rf_grid.cv_results_['mean_test_score']

# Manual fit stacking

In [ ]:
class StackBaseTransformer(BaseEstimator, TransformerMixin):
    estimators = [("logreg", logreg_pipeline),
                  ("dtree", dtree_pipeline),
                  ("rf", random_forest_pipeline),
                  ("cb", catboost_pipeline),
                  ("knn", knn_pipeline),
                  ("nb", nb_pipeline),
                  ("svc", svc_pipeline),
                  ("xgb", xgb_pipeline),
                 ]
    
    def fit(self, X, y=None):
        for name, model in self.estimators:
            model.fit(X, y)

    def transform(self, X):
        newX = pd.DataFrame()
        for name, model in self.estimators:
            newX[name] = model.predict_proba(X)[:, 1].round(2)
        return newX

SBT = StackBaseTransformer()
SBT.fit(X, y)
X_sbt = SBT.transform(X)

In [ ]:
pd.concat([y, X_sbt, X], axis=1)

In [ ]:
sns.heatmap(X_sbt.corr(), annot=True)

# choose model for private data

In [ ]:
best_model = stacking
best_model.fit(X, y) # refit model on whole data

X_private = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/test.csv')
submission = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/submission.csv')
submission['Churn'] = best_model.predict_proba(X_private)[:, 1]
submission.to_csv('./my_submission.csv', index=False, columns=["Id", "Churn"])